In [ ]:
import ReadRawMHD as rR
import test_delete_ as test
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import time
import os
t1 = time.time()

In [ ]:
# # # # # Input # # # # #
# Input imaging data
model_code = '00_Pilot3'  # FEA model name
sample_code = '00_Pilot3'  # contains number and sample name, e.g. 00_Pilot3.
screw = 'T'  # T or P, site from experiment. Two input files will be generated for each site
file_bone = 'XCT_Icotec_S130672_L5_intact_planned.mhd'  # gray value bone ct scan
file_inst = 'ICOTEC_S130672_L5_implants_XCTres.mhd'  # segmented screw scan

In [ ]:
# # # # # Input # # # # #
# Input FEA data
file_stl = model_code + '_mesh.stl'
file_dummyMesh = model_code + '_mesh.inp'  # contains nodes and elements
#file_dummyInp = 'Pilot3_mesh.inp'

In [ ]:
# Specify file locations
sample_no, sample = sample_code.split('_')  # sample name and number
path_project = '/home/biomech/Documents/01_Icotec/'  # General project folder
path_ct = path_project + '01_Experiments/02_Scans/' + sample + '/04_Registered/'  # Folder of CT data
path_fea = path_project + '02_FEA/99_Tests/' + sample + '/' + sample_code + '/'  # Folder of FEA files
# Include general path for mesh/template later
info = sample + screw + '.txt'  # .txt file containing info about landmarks. Specific structure

In [ ]:
# Write mesh input file
test.write_mesh(path_fea + sample_code + '_model.inp', path_fea + file_dummyMesh)  # Original input file, path for mesh.inp

In [ ]:
# Load data
imD = rR.load_itk(path_ct + file_inst)  # screw image
with open(path_ct + info, 'r') as f:
    content = f.read()
ori = content.split('Origin: ')[1].split('\n')[0]
ori = np.array([int(ori.split(' ')[0]), int(ori.split(' ')[1]), int(ori.split(' ')[2])])
p1P = content.split('positive x axis: ')[1].split('\n')[0]
p1P = np.array([int(p1P.split(' ')[0]), int(p1P.split(' ')[1]), int(p1P.split(' ')[2])])
p1N = content.split('negative x axis: ')[1].split('\n')[0]
p1N = np.array([int(p1N.split(' ')[0]), int(p1N.split(' ')[1]), int(p1N.split(' ')[2])])

In [ ]:
# Plot screws
plt.imshow(np.sum(imD[0], axis=1))  # xray with both screw on it, needed to define ROI for screw axis extraction

In [ ]:
# # # # # Input # # # # #
# Define screw vector
v3 = []
if screw == 'T' or screw == '':
    lineT = rR.axis3D(imD[0], 670, 1100, 'x')  # if starting from screw tip towards head, add (-) to v3
    v3 = -lineT.vector  # z-axis = screw axis, found by function. Specify +/-!!
elif screw == 'P':
    lineT = rR.axis3D(imD[0], 0, 600, 'x')  # if starting from screw tip towards head, add (-) to v3
    v3 = lineT.vector  # z-axis = screw axis, found by function. Specify +/-!!
del imD

In [ ]:
# Compute matrix
v2 = np.cross(v3, p1P-p1N)/np.linalg.norm(np.cross(v3, p1P-p1N))  # y-axis
v1 = np.cross(v2, v3)  # x-axis
M = np.vstack((np.append(v1, ori[0]), np.append(v2, ori[1]), np.append(v3, ori[2]), np.array([0, 0, 0, 1])))
print(M)

In [ ]:
# Check coordinate system
# z-axis (blue) should point from screw head (origin) towards screw tip (dot)
# y-axis (green) should be perpendicular to screw-rotAxis plane and point upwards (against loading direction)
# x-axis (red): right-handed coordinate system
%matplotlib notebook
plt.figure()
ax = plt.axes(projection='3d')
fact = 100
for i in range(-0, 5):
    ax.scatter3D(ori[0]+i*fact*v1[0], ori[1]+i*fact*v1[1], ori[2]+i*fact*v1[2], c='r', alpha=1)
    ax.scatter3D(ori[0]+i*fact*v2[0], ori[1]+i*fact*v2[1], ori[2]+i*fact*v2[2], c='g', alpha=1)
    ax.scatter3D(ori[0]+i*fact*v3[0], ori[1]+i*fact*v3[1], ori[2]+i*fact*v3[2], c='b', alpha=1)
ax.scatter3D(p1P[0], p1P[1], p1P[2], c='b')
ax.scatter3D([0, 1500], [0, 1500], [0, 1500], alpha=0)
ax.scatter3D(686, 480, 177, c='k', alpha=1)  # Ti tip
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

In [ ]:
bone = {}
stl = path_fea + file_stl

In [ ]:
dummyMesh = path_fea + file_dummyMesh

In [ ]:
bone = test.readInpBoneDummy(bone, dummyMesh)  # Read bone mesh from abaqus. Read elements, nodes

In [ ]:
bone = test.boneMeshMask(bone, path_fea, stl, 0.0607, sample_code + screw + '_mask.mhd')  # Create mask from abaqus bone mesh

In [ ]:
bone = test.load_BVTVdata(bone, path_ct + file_bone)

In [ ]:
# Read mask
imMask = sitk.ReadImage(path_fea + sample_code + screw + '_mask.mhd')
imMask_np = np.transpose(sitk.GetArrayFromImage(imMask), [2, 1, 0])
bone["GreyImage"].SetOrigin([0, 0, 0])
#im_grey_gauss=sitk.GetImageFromArray(np.transpose(bone['GreyImage'],[2,1,0]))  # save gauss image
#sitk.WriteImage(im_grey_gauss,path_ct + sample_code + 'Gauss.mhd')
# Define rotation and translation
[theta1, theta2, theta3] = rR.rotation_angles_from_matrix(M[:3, :3], 'zyx')
theta2 = -theta2 + np.pi  # +pi because arc-tan no unique solution. WHY NEGATIVE, because mask[:, ::, :] reshaped?
center = np.array([imMask_np.shape[0]/2, imMask_np.shape[1]/2, 0]) * imMask.GetSpacing()
trans = M[:3, 3] * imMask.GetSpacing() - center

In [ ]:
# Write transformation file
f = open(path_fea + 'transformation_' + sample_code + screw + '.tfm', "w")
f.write(
    "#Insight Transform File V1.0\n"
    "#Transform 0\n"
    "Transform: CompositeTransform_double_3_3\n"
    "#Transform 1\n"
    "Transform: Euler3DTransform_double_3_3\n"
    "Parameters:  " + f'{theta1}' + " " + f'{theta2}' + " " + f'{theta3}'
    + " " + f'{trans[0]}' + " " + f'{trans[1]}' + " " + f'{trans[2]}' + "\n"  # transformation
    "FixedParameters: " + f'{center[0]}' + " " + f'{center[1]}' + " " + f'{center[2]}' + " 0\n")  # CoR
f.close()

In [ ]:
# Apply transformation to mask using the inverse transformation
bone['Transform'] = sitk.ReadTransform(path_fea + 'transformation_' + sample_code + screw + '.tfm')
bone['Transform_inv'] = bone['Transform'].GetInverse()
imMask_trans = sitk.Resample(imMask, bone['GreyImage'], bone['Transform_inv'], sitk.sitkNearestNeighbor, 0.0, bone['GreyImage'].GetPixelID())
# Delete some files / variables to save memory
os.remove(path_fea + sample_code + screw + '_mask.mhd')  #---here mask removing test)
os.remove(path_fea + sample_code + screw + '_mask.raw')  #---here mask removing test)
img_seg=sitk.GetImageFromArray(np.transpose(bone['BVTVscaled'],[2,1,0]))
img_seg.SetOrigin(bone["GreyImage"].GetOrigin())
img_seg.SetSpacing(bone["GreyImage"].GetSpacing())
sitk.WriteImage(img_seg,path_fea+'Test_seg.mhd')
del bone['GreyImage']

In [ ]:
# BVTV segmentation / calibration HR-pQCT to uCT
bone['MASK_array_T'] = np.transpose(sitk.GetArrayFromImage(imMask_trans), [2, 1, 0])
# scaling factor/intercept from Schenk et al. 2022, has to be discussed w Ph

In [ ]:
elsets = path_fea + sample_code + 'elsets' + screw + '.inp'
materials = path_fea + sample_code +  'materials' + screw + '.inp'
rR.blockPrint()
test.HFE_mapping_trans(bone, elsets, materials)
rR.enablePrint()

In [ ]:
sliceNo = 460
plt.imshow(bone['BVTVscaled'][:, sliceNo, :] + bone['MASK_array_T'][:, sliceNo, :])

plt.show()

In [ ]:
# Write final input file
template = path_fea + sample_code + '_model.inp'  # orig is template file
test.HFE_inp_creator(template, elsets, materials, path_fea + sample_code + screw + '_P', 'PEEK')
test.HFE_inp_creator(template, elsets, materials, path_fea + sample_code + screw + '_T', 'Ti')
# Clean up folder
os.remove(elsets)
os.remove(materials)

In [ ]:
print('Execution time: ' + str(int((time.time()-t1)/60)) + ' min '+ str(round(np.mod(time.time()-t1, 60), 1)) + ' sec.')